In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

## Extract CSVs into DataFrames

In [2]:
# Extract happiness rankings CSV into DataFrame
happy_file = "resources/2019_world_happiness_rankings.csv"
happy_df = pd.read_csv(happy_file, encoding='UTF-8')
happy_df.head()

Overall rank Country or region  Score  GDP per capita  Social support  \
0             1           Finland  7.769           1.340           1.587   
1             2           Denmark  7.600           1.383           1.573   
2             3            Norway  7.554           1.488           1.582   
3             4           Iceland  7.494           1.380           1.624   
4             5       Netherlands  7.488           1.396           1.522   

   Healthy life expectancy  Freedom to make life choices  Generosity  \
0                    0.986                         0.596       0.153   
1                    0.996                         0.592       0.252   
2                    1.028                         0.603       0.271   
3                    1.026                         0.591       0.354   
4                    0.999                         0.557       0.322   

   Perceptions of corruption  
0                      0.393  
1                      0.410  
2                      0.341  
3                      0.118  
4                      0.298

In [3]:
# Extract university rankings CSV into DataFrame
university_file = "resources/2019_QS_world_university_rankings.csv"
university_df = pd.read_csv(university_file, encoding='ISO-8859-1')
university_df.head()

2019 2018                              Institution Name         Country  \
0    1    1   MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT)   United States   
1    2    2                           STANFORD UNIVERSITY   United States   
2    3    3                            HARVARD UNIVERSITY   United States   
3    4    4  CALIFORNIA INSTITUTE OF TECHNOLOGY (CALTECH)   United States   
4    5    6                          UNIVERSITY OF OXFORD  United Kingdom   

  Classification Size Focus Research Intensity  Age Status  \
0                   M    CO                 VH    5      B   
1                   L    FC                 VH    5      B   
2                   L    FC                 VH    5      B   
3                   S    CO                 VH    5      B   
4                   L    FC                 VH    5      A   

  Academic Reputation Score  ... Employer Reputation Rank  \
0                       100  ...                        4   
1                       100  ...                        5   
2                       100  ...                        1   
3                      98.7  ...                       72   
4                       100  ...                        3   

  Faculty Student Score Faculty Student Rank Citations per Faculty Score  \
0                   100                   17                        99.8   
1                   100                   16                          99   
2                  99.3                   40                        99.8   
3                   100                    4                         100   
4                   100                    7                          83   

  Citations per Faculty Rank International Faculty Score  \
0                          8                         100   
1                         13                        99.8   
2                          9                        92.1   
3                          5                        96.8   
4                         56                        99.6   

  International Faculty Rank International Students Score  \
0                         41                         95.5   
1                         56                         70.5   
2                        151                         75.7   
3                        115                         90.3   
4                         63                         98.8   

  International Students Rank Overall Score  
0                          77           100  
1                         190          98.6  
2                         164          98.5  
3                         103          97.2  
4                          43          96.8  

[5 rows x 22 columns]

## Clean Up of University Rankings

In [4]:
# Remove unnecessary columns
university_df = university_df.drop(['Focus', 'Research Intensity', 'Age', 'Status'], axis=1)

# Drop universities that were unranked in 2019
university_df = university_df.dropna(subset=['2019'])

In [5]:
# The dataset includes '=' at the end of ranking ties
university_df['2019'] = university_df['2019'].str.rstrip('=')
university_df['2018'] = university_df['2018'].str.rstrip('=')

In [6]:
# The dataset includes '+' at the end of the attribute ranks (i.e. International Faculty Rank of 601+)
university_df['Academic Reputation Rank'] = university_df['Academic Reputation Rank'].str.rstrip('+')
university_df['Employer Reputation Rank'] = university_df['Employer Reputation Rank'].str.rstrip('+')
university_df['Faculty Student Rank'] = university_df['Faculty Student Rank'].str.rstrip('+')
university_df['Citations per Faculty Rank'] = university_df['Citations per Faculty Rank'].str.rstrip('+')
university_df['International Faculty Rank'] = university_df['International Faculty Rank'].str.rstrip('+')
university_df['International Students Rank'] = university_df['International Students Rank'].str.rstrip('+')

In [7]:
# Because some rankings begin with a double space, we need to remove these characters
university_df['2019'] = university_df['2019'].str.strip()
university_df['2018'] = university_df['2018'].str.strip()

In [8]:
# The dataset includes ranges for higher rankings (i.e. 801-1000)
# Split the rankings from 2019 to only include lowest rank in the range
university_df['2019'] = university_df['2019'].str[:3]
university_df['2018'] = university_df['2018'].str[:3]

In [9]:
# When a score or rank was not taken for a university, the CSV notes it with either a '-' or an empty cell
# Convert all '-' cells into empty cells
university_df.replace({'-': np.nan}, inplace=True)

In [10]:
# Rename columns in both DataFrames
# Happiness rankings DataFrame
happy_df = happy_df.rename(columns = {
    'Overall rank': 'overall_rank',
    'Country or region': 'country',
    'Score': 'score',
    'GDP per capita': 'gdp_per_capita_score',
    'Social support': 'social_support_score',
    'Healthy life expectancy': 'healthy_life_expectancy_score',
    'Freedom to make life choices': 'freedom_life_choices_score',
    'Generosity': 'generosity',
    'Perceptions of corruption': 'perceived_corruption_score'
})

# University rankings DataFrame
university_df = university_df.rename(columns = {
    '2019': 'year_2019',
    '2018': 'year_2018',
    'Institution Name': 'institution_name',
    'Country': 'country',
    'Classification Size': 'class_size',
    'Academic Reputation Score': 'academic_rep_score',
    'Academic Reputation Rank': 'academic_rep_rank',
    'Employer Reputation Score': 'employer_rep_score',
    'Employer Reputation Rank': 'employer_rep_rank',
    'Faculty Student Score': 'faculty_student_score',
    'Faculty Student Rank': 'faculty_student_rank',
    'Citations per Faculty Score': 'faculty_citations_score',
    'Citations per Faculty Rank': 'faculty_citations_rank',
    'International Faculty Score': 'intl_faculty_score',
    'International Faculty Rank': 'intl_faculty_rank',
    'International Students Score': 'intl_students_score',
    'International Students Rank': 'intl_students_rank',
    'Overall Score': 'overall_score'})

In [11]:
# Set indices
university_df.set_index('institution_name', inplace=True)
happy_df.set_index('country', inplace=True)

In [12]:
# Display the cleaned Happiness DataFrame
happy_df.head()

overall_rank  score  gdp_per_capita_score  social_support_score  \
country                                                                        
Finland                 1  7.769                 1.340                 1.587   
Denmark                 2  7.600                 1.383                 1.573   
Norway                  3  7.554                 1.488                 1.582   
Iceland                 4  7.494                 1.380                 1.624   
Netherlands             5  7.488                 1.396                 1.522   

             healthy_life_expectancy_score  freedom_life_choices_score  \
country                                                                  
Finland                              0.986                       0.596   
Denmark                              0.996                       0.592   
Norway                               1.028                       0.603   
Iceland                              1.026                       0.591   
Netherlands                          0.999                       0.557   

             generosity  perceived_corruption_score  
country                                              
Finland           0.153                       0.393  
Denmark           0.252                       0.410  
Norway            0.271                       0.341  
Iceland           0.354                       0.118  
Netherlands       0.322                       0.298

In [13]:
# Display the cleaned DataFrame
university_df.head()

year_2019 year_2018  \
institution_name                                                   
MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT)          1         1   
STANFORD UNIVERSITY                                  2         2   
HARVARD UNIVERSITY                                   3         3   
CALIFORNIA INSTITUTE OF TECHNOLOGY (CALTECH)         4         4   
UNIVERSITY OF OXFORD                                 5         6   

                                                     country class_size  \
institution_name                                                          
MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT)    United States          M   
STANFORD UNIVERSITY                            United States          L   
HARVARD UNIVERSITY                             United States          L   
CALIFORNIA INSTITUTE OF TECHNOLOGY (CALTECH)   United States          S   
UNIVERSITY OF OXFORD                          United Kingdom          L   

                                             academic_rep_score  \
institution_name                                                  
MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT)                 100   
STANFORD UNIVERSITY                                         100   
HARVARD UNIVERSITY                                          100   
CALIFORNIA INSTITUTE OF TECHNOLOGY (CALTECH)               98.7   
UNIVERSITY OF OXFORD                                        100   

                                             academic_rep_rank  \
institution_name                                                 
MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT)                  3   
STANFORD UNIVERSITY                                          5   
HARVARD UNIVERSITY                                           1   
CALIFORNIA INSTITUTE OF TECHNOLOGY (CALTECH)                21   
UNIVERSITY OF OXFORD                                         4   

                                             employer_rep_score  \
institution_name                                                  
MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT)                 100   
STANFORD UNIVERSITY                                         100   
HARVARD UNIVERSITY                                          100   
CALIFORNIA INSTITUTE OF TECHNOLOGY (CALTECH)               81.2   
UNIVERSITY OF OXFORD                                        100   

                                             employer_rep_rank  \
institution_name                                                 
MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT)                  4   
STANFORD UNIVERSITY                                          5   
HARVARD UNIVERSITY                                           1   
CALIFORNIA INSTITUTE OF TECHNOLOGY (CALTECH)                72   
UNIVERSITY OF OXFORD                                         3   

                                             faculty_student_score  \
institution_name                                                     
MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT)                    100   
STANFORD UNIVERSITY                                            100   
HARVARD UNIVERSITY                                            99.3   
CALIFORNIA INSTITUTE OF TECHNOLOGY (CALTECH)                   100   
UNIVERSITY OF OXFORD                                           100   

                                             faculty_student_rank  \
institution_name                                                    
MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT)                    17   
STANFORD UNIVERSITY                                            16   
HARVARD UNIVERSITY                                             40   
CALIFORNIA INSTITUTE OF TECHNOLOGY (CALTECH)                    4   
UNIVERSITY OF OXFORD                                            7   

                                             faculty_citations_score  \
institution_name                                                       
MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT)              

## Create database connection

In [ ]:
# Require user to input postgres password
password = input("Please input your postgres password: ")

# Create connection
connection_string = f"postgres:{password}@localhost:5432/etl_project_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

## Load DataFrame into database

In [ ]:
happy_df.to_sql(name='world_happiness', con=engine, if_exists='append', index=True)

In [ ]:
university_df.to_sql(name='university_rankings', con=engine, if_exists='append', index=True)

## Export DataFrames to csv Files

In [ ]:
# saving the dataframe to csv files
happy_df.to_csv(r'Results/final_world_happy_data.csv', header=True, index=False)
university_df.to_csv(r'Results/final_univ_rank_data.csv', header=True, index=False)